In [1]:
import wikipedia
import pandas as pd
import spacy
import torch
from tqdm import tqdm

from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16 = True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [13]:
df_es = pd.read_json(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\test_ds\v1\mushroom.es-tst.v1.jsonl', lines=True) #Cambiar para directorio local
df_es.head(3)

,id,lang,model_input,model_output_text,model_id,model_output_tokens,model_output_logits
0,tst-es-1,ES,¿En qué equipos de la NHL jugó Bert Olmstead d...,Bert Olmstead jugó para los Montreal Canadiens...,Iker/Llama-3-Instruct-Neurona-8b-v2,"[B, ert, ĠOl, m, stead, Ġjug, Ã³, Ġpara, Ġlos,...","[26.1876564026, 30.4659004211, 27.9287395477, ..."
1,tst-es-2,ES,"¿Cuántos habitantes tiene Forstescue, Misuri?","Según el Bureau del Censo de Estados Unidos, l...",Iker/Llama-3-Instruct-Neurona-8b-v2,"[Seg, Ãºn, Ġel, ĠBureau, Ġdel, ĠC, enso, Ġde, ...","[21.147600174, 28.2901725769, 20.9695224762, 1..."
2,tst-es-3,ES,¿En dónde se han encontrado restos de la balle...,Los restos fósiles de la ballena picuda de Bah...,Qwen/Qwen2-7B-Instruct,"[Los, Ġrest, os, Ġf, Ã³s, iles, Ġde, Ġla, Ġbal...","[20.4515380859, 20.4586791992, 31.832113266, 1..."


In [2]:
# Estas tres partes están bastante bien.
# El problema siento que radica en la siguiente celda donde recuperamos el resumen de cada página en vez de las secciones relevantes.
def noun_list(a, lang):
    """
    Filtra la pregunta y obtiene las PoST relevantes.
    
    a = list; Lista de preguntas del dataset
    lang = 'es' or 'en'; Idioma a trabajar
    """
    if lang == 'es':
        post_spacy = spacy.load("es_core_news_sm")
    else:
        post_spacy = spacy.load("en_core_web_sm")
    noun_list = []
    nums = ['0','1','2','3','4','5','6','7','8','9']

    for _ in a:
        doc = post_spacy(_)
        sub_noun = []
        for token in doc:
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "NUM":
                sub_noun.append(token.text)
            if token.pos_ == "ADJ" and token.text[0] in nums:
                sub_noun.append(token.text)
        if not sub_noun:
            noun_list.append(_.split())
        else:
            noun_list.append(sub_noun)
    return noun_list

def keyword_por_preg(n_list):
    """
    Junta lista de PoST previo a pasarlo por el API de Wikipedia.
    
    n_list = list; Obtenida de la función noun_list().
    """
    keyword_list = []
    for i in n_list:
        keyword = ''
        for j in i:
            keyword = keyword + j + ' '
        #if keyword == '': # CORRECCIÓN
        #    keyword_list.append(i) # CORRECCIÓN
        else:
            keyword_list.append(keyword)
    return keyword_list

def get_wikipage(text, lang, page_total):
    """
    Regresa las n páginas de Wikipedia más relevantes al query

    text = str; Texto proveniente de la función keyword_por_preg()
    lang = 'es' or 'en'; Lenguaje necesario para wikipedia
    page_total = int; Cantidad de páginas a regresar
    """
    if lang == 'es':
        wikipedia.set_lang('es')
    if lang == 'en':
        wikipedia.set_lang('en')
    page_title = wikipedia.search(text, results = page_total)
    return page_title

In [3]:
preg = ['In which film does James Bond drive an Aston Martin V12 Vanquish?']
a = noun_list(preg, 'en')
print(a)
b = keyword_por_preg(a)
print(b)
c = get_wikipage(b, 'en', 3)
print(c)

resumen = ''
for _ in c:
    page = wikipedia.WikipediaPage(_)
    summary = page.summary
    resumen += summary + '--'

print(resumen)

[['film', 'James', 'Bond', 'Aston', 'Martin', 'V12', 'Vanquish']]
['film James Bond Aston Martin V12 Vanquish ']
['Aston Martin Vanquish', 'Aston Martin Vanquish (2012)', 'Aston Martin DBS (2007)']
The Aston Martin Vanquish is a grand tourer introduced by British luxury automobile manufacturer Aston Martin in 2001 as a successor to the Aston Martin Virage (1993).
The Aston Martin V12 Vanquish, designed by Ian Callum and unveiled at the 2001 Geneva Motor Show, was produced from 2001 to 2007 as the flagship of the marque. A concept car, known as "Project Vantage" and the first Aston Martin design wholly designed by Callum, was built to display the company's vision for a future sports car that could represent Aston Martin's aspirations after the discontinuation of the Virage-based Vantage. The concept car evolved directly into the V12 Vanquish, and featured an advanced carbon fibre and alloy structure, Aston Martin's most powerful V12 engine, and host of new technologies.  A specially mod

In [8]:
def get_content(wiki_names):
    """
    Regresa la lista content con las páginas filtradas y segmentadas.

    wiki_names = list ; Lista de los nombres de páginas de Wikipedia. Extraído del dataset. 
    """
    p_content = []
    content = []
    
    for _ in wiki_names:
        text = ''
        try:
            page = wikipedia.WikipediaPage(_)
            textaux = page.content.replace('\n', '')
            textaux = textaux.replace('\t', '')
            text += '  ' + textaux
        except wikipedia.exceptions.DisambiguationError:
            print("Error")
        #text = page.content.replace('\n', '')
        #text = text.replace('\t', '')
        p_content.append(text)

    for _ in p_content:
        _ = _.split("===")
        texto = [i.replace('=', '') for i in _]
        #texto = [i.replace('\ ', '') for i in _]
        aux = []
        for j in texto:
            if len(j) > 60:
                aux.append(j)
        content.append(aux)

    return content

def embeddings_t1(seg_txt):
    """
    Obtenemos los embeddings de cada elemento de seg_txt

    seg_txt = list ; Instancia única de content_list (content_list[i] for i in N)
    """
    len_list = []
    for _ in seg_txt:
        for j in _:
            len_list.append(len(j))
    max_len = max(len_list)
    embedding_list = []

    if max(len_list) < 8192: # Verificamos que el texto no sea tan largo (TENEMOS QUE SEGMENTAR DE MEJOR MANERA LOS TEXTOS DE WIKISEX)
        max_len = max(len_list)
    else:
        max_len = 8192
    emb_list = []
    
    for _ in seg_txt:
        for j in _:
            embs = model.encode(
                j,
                batch_size = 12,
                max_length = max_len,
            )["dense_vecs"]
            embedding_list.append(embs)
    return embedding_list


def get_content_embs(content_list):
    """
    Genera la lista de content_embeddings a partir de la lista "content" correspondiente.

    content_list  = list ; lista obtenida 
    """
    content_embs = [embeddings_t1(_) for _ in content_list]
    return content_embs

In [15]:
# Preg_Emb
def similarity(t1, t2):
    return t1 @ t2

def q_emb(pregunta):
    """
    Genera embedding de una pregunta.

    pregunta = dataset["model_input"][i] ; Pregunta a vectorizar
    """
    preg_emb = model.encode(
        pregunta,
        batch_size=12,
        max_length = 512,
    )['dense_vecs']
    return preg_emb

In [16]:
def resumen_topn(texto, embeddings, pregunta, top_n):
    """
    texto = list ; lista obtenida de get_content()
    embeddings = list ; lista obtenida de get_content_embs()
    pregunta = BGE Embedding ; Correspondiente a una pregunta del dataset.
    top_n = int ; Cantidad de vectores usados para el resumen.
    """
    full = []
    resumen = ''
    for i in range(len(texto)):
        aux = list(zip(texto[i], embeddings[i]))
        for _ in aux:
            full.append(_)

    simi_list = [(similarity(pregunta, _[1]), _[0]) for _ in full]
    simi_list.sort(reverse=True)

    for i in simi_list[:top_n]:
        resumen += '---' + i[1]
    return resumen

In [14]:
%%time
a1 = get_content(c)
b1 = embeddings_t1(a1[0])
b2 = embeddings_t1(a1[1])
b3 = embeddings_t1(a1[2])

CPU times: total: 14min 25s
Wall time: 14min 29s


In [18]:
emb_list = [b1, b2, b3]
ques = q_emb('In which film does James Bond drive an Aston Martin V12 Vanquish?')
resumen = resumen_topn(a1, emb_list, ques, 3)

In [19]:
resumen

'---The interior of the DBS is a blend of carbon fibre, Alcantara, leather, wood, stainless steel and aluminium surfaces, depending on the buyer\'s specified options. The door panels are capped with carbon fibre or leather, and utilise carbon fibre door pulls. The fascia is, as standard, matrix alloy and iridium silver centre console or, as an optional extra, piano black fascia and centre console. To achieve even greater weight savings, the carpet has a special lightweight carbon weave. The car is started by means of the "Emotion Control Unit", which was initially developed especially for the DBS but became available for the DB9 and the V8 Vantage as well. The key is made from stainless steel and glass and is inserted into a special slot in the dashboard.== Film appearances ==The DBS was first seen in the 2006 James Bond film Casino Royale, the first film in which Bond was played by Daniel Craig, as a result of Eon Production\'s desire to tie the new Bond actor to the franchise heritag

In [23]:
from transformers import pipeline
model_instruct = "meta-llama/Llama-3.2-3B-Instruct"
pipe1 = pipeline(
    "text-generation",
    model = model_instruct,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
)

def llama_gen(ques, context, lang):
    prompt = f"""
        You are a bot that answers trivia questions.
        Be brief, answer in short sentences highlighting important information.
        If the given text doesn't answer the question, answer as truthfully as you can with your own information.
        
        This is the trivia question you need to answer:
        {ques}
        
        This is the text that you should use:
        {context}
    """
    
    prompt_es = f"""
        Eres un bot que contesta preguntas de trivia.
        Se conciso, contesta en oraciones cortas y resalta información importante.
        Si el contexto dado no te permite contestar la pregunta, contesta de la manera más verídica con tu propia información.

        Esta es la pregunta que tienes que contestar:
        {ques}

        Este es el contexto que debes usar:
        {context}
    """

    if lang == 'es':
        messages = [
            {"role": "system", "content": "Eres un chatbot que responde preguntas con alta veracidad dado un texto. Si el texto no da la información correcta, contesta con tu propia información."},
            {"role": "user", "content": prompt_es}
        ]
    else:
        messages = [
            {"role": "system", "content": "You are a chatbot that responds general knowledge questions with high fidelity based on a given text. If the text doesn't give the correct information, answer with your own information."},
            {"role": "user", "content": prompt}
        ]
        
    outputs = pipe1(
        messages,
        max_new_tokens = 256,
    )
    return outputs[0]["generated_text"][-1]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [26]:
llama = llama_gen('In which film does James Bond drive an Aston Martin V12 Vanquish?', resumen, 'en')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [27]:
print(llama)

{'role': 'assistant', 'content': 'James Bond drives an Aston Martin DBS in the films "Casino Royale" (2006) and "Quantum of Solace" (2008).'}


In [65]:
def ds_procesamiento(dataset, lang):
    """
    Regresa un dataset bien formateado, acá chido para todo el procesamiento.
    
    dataset = pd.DataFrame ; El dataset mismo, así encuerado.
    lang = str ; 'es' or 'en' Para determinar el idioma.
    """
    a = keyword_por_preg(noun_list(dataset["model_input"], lang))
    dataset["output_filtrado"] = a

    a_set = list(set(a))
    dic = {}
    for _ in a_set:
        aux = get_wikipage(_, lang, 3) # ¿Qué pasa si no encuentra páginas adecuadas?
        if not aux:
            wiki_aux = wikipedia.suggest(_)
            if wiki_aux != None:
                aux = get_wikipage(wiki_aux, lang, 3)
            else:
                print(_)
        dic[f"{_}"] = aux

    aux1 = []
    for i in range(len(dataset["output_filtrado"])):
        if dataset["output_filtrado"][i] in dic:
            aux1.append(dic[dataset["output_filtrado"][i]])
    dataset["Wiki Asociado"] = aux1
    return dataset

In [66]:
%%time
aux_ds = ds_procesamiento(df_es, 'es')
aux_ds

prueba estadounidense Hermann Frazier 
disciplina Jarrin Solomon 
tipo animal Isolobodon portoricensis 
CPU times: total: 1.77 s
Wall time: 54.8 s


,id,lang,model_input,model_output_text,model_id,model_output_tokens,model_output_logits,output_filtrado,Wiki Asociado
0,tst-es-1,ES,¿En qué equipos de la NHL jugó Bert Olmstead d...,Bert Olmstead jugó para los Montreal Canadiens...,Iker/Llama-3-Instruct-Neurona-8b-v2,"[B, ert, ĠOl, m, stead, Ġjug, Ã³, Ġpara, Ġlos,...","[26.1876564026, 30.4659004211, 27.9287395477, ...",equipos NHL Bert Olmstead carrera,"[Bert Olmstead, California Golden Seals]"
1,tst-es-2,ES,"¿Cuántos habitantes tiene Forstescue, Misuri?","Según el Bureau del Censo de Estados Unidos, l...",Iker/Llama-3-Instruct-Neurona-8b-v2,"[Seg, Ãºn, Ġel, ĠBureau, Ġdel, ĠC, enso, Ġde, ...","[21.147600174, 28.2901725769, 20.9695224762, 1...",habitantes Forstescue Misuri,"[Fortescue (Misuri), Condado de Holt (Misuri),..."
2,tst-es-3,ES,¿En dónde se han encontrado restos de la balle...,Los restos fósiles de la ballena picuda de Bah...,Qwen/Qwen2-7B-Instruct,"[Los, Ġrest, os, Ġf, Ã³s, iles, Ġde, Ġla, Ġbal...","[20.4515380859, 20.4586791992, 31.832113266, 1...",restos ballena Bahamonde Mesoplodon,[Mesoplodon traversii]
3,tst-es-4,ES,¿Quién ganó el premio BAFTA al mejor actor de ...,"Según la información disponible, el premio BAF...",meta-llama/Meta-Llama-3-8B-Instruct,"[Seg, Ãºn, Ġla, ĠinformaciÃ³n, Ġdisponible, ,,...","[20.6939048767, 28.1913757324, 23.7558517456, ...",premio BAFTA actor reparto año 2000 película,"[Premios BAFTA, Premios del Sindicato de Actor..."
4,tst-es-5,ES,¿Cuántas medidas disciplinares se aprobaron en...,"El Segundo Concilio de Nicea, también conocido...",Iker/Llama-3-Instruct-Neurona-8b-v2,"[El, ĠSeg, undo, ĠConc, ilio, Ġde, ĠNice, a, ,...","[19.8374900818, 21.1202754974, 34.8290023804, ...",medidas consejo Nicea año 787,"[Concilio de Fráncfort, Primeros siete concili..."
...,...,...,...,...,...,...,...,...,...
147,tst-es-148,ES,¿Cuántas veces ha sido seleccionado Domantas S...,Hasta la fecha de mi última actualización en o...,Qwen/Qwen2-7B-Instruct,"[H, asta, Ġla, Ġfecha, Ġde, Ġmi, ĠÃºltima, Ġac...","[20.6600112915, 27.0722904205, 18.8246574402, ...",veces Domantas Sabonis hijo Arvydas Sabonis Ga...,[Domantas Sabonis]
148,tst-es-149,ES,¿Qué líneas de metro puedo tomar desde la esta...,"Desde la estación de République en París, pued...",Iker/Llama-3-Instruct-Neurona-8b-v2,"[Desde, Ġla, Ġest, aciÃ³n, Ġde, ĠRÃ©, pub, liq...","[21.195646286, 24.2160949707, 23.9696769714, 3...",líneas metro estación République París,"[Estación de République, Línea 9 del Metro de ..."
149,tst-es-150,ES,¿A qué altura se localiza el municipio de Yush...,"El municipio de Yushu, ubicado en la provincia...",Qwen/Qwen2-7B-Instruct,"[El, Ġmunicip, io, Ġde, ĠY, ush, u, ,, Ġubic, ...","[27.9522533417, 21.2112312317, 33.8999862671, ...",altura municipio Yushu Qinghai China,"[Yushu, Yangtsé]"
150,tst-es-151,ES,¿En qué años se produjo la primera generación ...,La primera generación del Aston Martin Vanquis...,Qwen/Qwen2-7B-Instruct,"[La, Ġprimera, Ġgener, aciÃ³n, Ġdel, ĠAston, Ġ...","[27.9550552368, 24.7381954193, 25.8791179657, ...",años generación Aston Martin Vanquish,"[Aston Martin Vanquish, Aston Martin DB9, Asto..."


In [104]:
def get_res(dataset):
    """
    Genera el resumen por cada pregunta a partir de sus embeddings

    dataset = pd.DataFrame ; Formateado usando ds_procesamiento()
    """
    resumen_list = []
    for i in tqdm(range(len(dataset["Wiki Asociado"]))):
        temp_wiki = dataset["Wiki Asociado"][i]
        preg = dataset["model_input"][i]

        content = get_content(temp_wiki)
        content_embs = get_content_embs(content)

        p_emb = q_emb(preg)
        res = resumen_topn(content, content_embs, p_emb, 5)
        resumen_list.append(res)
    return resumen_list

In [106]:
%%time
hola = get_res(aux_ds)

  3%|▎         | 4/152 [43:05<26:34:14, 646.32s/it] 


ConnectionError: HTTPConnectionPool(host='es.wikipedia.org', port=80): Max retries exceeded with url: /w/api.php?prop=info%7Cpageprops&inprop=url&ppprop=disambiguation&redirects=&titles=Concilio+de+Fr%C3%A1ncfort&format=json&action=query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D4E069EF70>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder'))

________

In [107]:
file = open(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\ContextoRAG.txt','w')
for i in hola:
	file.write(i+"\n")
file.close()

NameError: name 'hola' is not defined